# Setup Gym Environment

```py
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05 # when to give up
my_config["start_obs_capture"] = 0.05 # when to capture 
my_config["time_step_timeout_factor"] = 1.0 # how late is OK
my_config["act_buf_len"] = 3 # how many past actions
my_config["reset_act_buf"] = True # resect action buffer on reset
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2
```

This section needs to be setup for any method

In [1]:
debugAsGym = False

from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 312
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'img_hist_len': 3,
  'modelMode': 4,
  'agent' : 'A3C',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 2, # 1 is High Speed Ring, 2 is 0-400m, 
}

In [2]:
if debugAsGym:
    env = gymnasium.make("real-time-gym-v1", config=my_config)

In [3]:
if debugAsGym:
    obs,info = env.reset()

In [4]:
if debugAsGym:
    obs, info = env.reset()
    env.render()
    print("Obs_actual (shape)")
    [print(o.shape) for o in obs]

In [5]:
if debugAsGym:
    print("Obs_space (shape)")
    [print(o.shape) for o in env.observation_space]

In [6]:
if debugAsGym:
    import pprint
    obs, info = env.reset()
    obs, rew, terminated, truncated, info = env.step(env.action_space.sample())
    obs_space = env.observation_space
    while not (terminated or truncated):
        act = env.action_space.sample()
        obs, rew, terminated, truncated, info = env.step(act)
        env.render()


# Register the environment in a way that RLlib is happy

In [7]:
if not debugAsGym:
    def env_creator(env_config):
        env = gymnasium.make("real-time-gym-v1", config=env_config)
        return env  # return an env instance

    from ray.tune.registry import register_env
    register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [8]:
if not debugAsGym:
    from ray.rllib.algorithms.a3c import A3CConfig

    algo = (
        A3CConfig()
        .resources(
            num_gpus=1
            )
        .rollouts(
            num_rollout_workers=1,
            enable_connectors=True,
            batch_mode="truncate_episodes",
            )
        .framework("torch")
        .environment(
            env="gt-rtgym-env-v1",
            disable_env_checking=True,
            render_env=False,
            )
        .training(
            #lr=tune.grid_search([0.01, 0.001, 0.0001]))
            #lambda_=0.95,
            #gamma=0.99,
            #sgd_minibatch_size=128,
            train_batch_size=256,
            #num_sgd_iter=8,
            #clip_param=0.2,
            #model={"fcnet_hiddens": [8, 8]},
        )
        .build()
    )

2023-05-16 22:13:00,284	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(RolloutWorker pid=10180) GT Real Time instantiated
(RolloutWorker pid=10180) GT AI Server instantiated for rtgym
(RolloutWorker pid=10180) simple reward system
(RolloutWorker pid=10180) starting up on localhost port 9999
(RolloutWorker pid=10180) Waiting for a connection
(RolloutWorker pid=10180) Connection from ('127.0.0.1', 51626)
(RolloutWorker pid=10180) reset triggered
(RolloutWorker pid=10180) reload save for track : 3


2023-05-16 22:13:14,153	INFO trainable.py:172 -- Trainable.setup took 16.299 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [9]:
if not debugAsGym:
    result = algo.train()

2023-05-16 22:13:18,251	WARNING worker.py:1986 -- Warning: More than 5000 tasks are pending submission to actor a21ea9a0fef691e9c66f033a01000000. To reduce memory usage, wait for these tasks to finish before sending more.


(RolloutWorker pid=10180) reset triggered
(RolloutWorker pid=10180) reload save for track : 3


(RolloutWorker pid=10180) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.1615634999996658
(RolloutWorker pid=10180)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=10180) reset triggered
(RolloutWorker pid=10180) reload save for track : 3


(RolloutWorker pid=10180) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.14784350000016389
(RolloutWorker pid=10180)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


In [10]:
algo.save()

'C:\\Users\\nadir/ray_results\\A3C_gt-rtgym-env-v1_2023-05-16_22-12-57e0y0lyli\\checkpoint_000001'

In [11]:
if not debugAsGym:
    N = 100

    for n in range(N):
        result = algo.train()
        print("Loop: ", n)
        if n % 20 == 0:
            print("Saved", n)
            algo.save()
    algo.save()

(RolloutWorker pid=10180) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.15176810000176033
(RolloutWorker pid=10180)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=10180) reset triggered
(RolloutWorker pid=10180) reload save for track : 3
(RolloutWorker pid=10180) reset triggered
(RolloutWorker pid=10180) reload save for track : 3


(RolloutWorker pid=10180) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.14148670000176367
(RolloutWorker pid=10180)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  0
Saved 0
(RolloutWorker pid=10180) reset triggered
(RolloutWorker pid=10180) reload save for track : 3


(RolloutWorker pid=10180) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.1435676000017736
(RolloutWorker pid=10180)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


...

(RolloutWorker pid=10180) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.15057609988707554
(RolloutWorker pid=10180)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=10180) reset triggered
(RolloutWorker pid=10180) reload save for track : 3


(RolloutWorker pid=10180) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.16109019988653017
(RolloutWorker pid=10180)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=10180) reset triggered
(RolloutWorker pid=10180) reload save for track : 3


(RolloutWorker pid=10180) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.17099519988641987
(RolloutWorker pid=10180)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


In [12]:
if not debugAsGym:
    N = 100

    for n in range(N):
        result = algo.train()
        print("Loop: ", n)
        if n % 20 == 0:
            print("Saved", n)
            algo.save()
    algo.save()

(RolloutWorker pid=10180) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 2826.5879091999996
(RolloutWorker pid=10180)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=10180) reset triggered
(RolloutWorker pid=10180) reload save for track : 3


(RolloutWorker pid=10180) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.12739359988700016
(RolloutWorker pid=10180)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=10180) reset triggered
(RolloutWorker pid=10180) reload save for track : 3


(RolloutWorker pid=10180) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.15831529988645343
(RolloutWorker pid=10180)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


...

(RolloutWorker pid=10180) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.15051249988573545
(RolloutWorker pid=10180)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=10180) reset triggered
(RolloutWorker pid=10180) reload save for track : 3
(RolloutWorker pid=10180) reset triggered
(RolloutWorker pid=10180) reload save for track : 3


(RolloutWorker pid=10180) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.17140409988678584
(RolloutWorker pid=10180)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  1


KeyboardInterrupt: 